In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [2]:
import sys
sys.path.append('../')

from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [3]:
#read in chicago validation labels
chicago_labels = gpd.read_file('../data/round-3/chicago-validation-labels-round3-230927.geojson')
chicago_labels = chicago_labels.drop(columns=['time_created'])
chicago_labels.head()

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,7958,1064,Addison,SurfaceProblem,41.976437,-87.979591,Vnk-r0opTiyjIyISxOMRUQ,None,1,f,cracks,None,POINT (-87.97959 41.97644)
1,Devon Snyder,7961,1064,Addison,SurfaceProblem,41.976780,-87.979561,w2tdzPaq9Rl7drhiXjPrvw,None,1,f,cracks,None,POINT (-87.97956 41.97678)
2,Devon Snyder,7965,1064,Addison,SurfaceProblem,41.977058,-87.979546,tfCdM8I4Mq5HZTZyleqk_g,None,1,f,height difference,None,POINT (-87.97955 41.97706)
3,Devon Snyder,7966,1064,Addison,SurfaceProblem,41.977795,-87.979553,dq_qwSfjMcwFLphaV8qzRw,None,1,f,cracks,None,POINT (-87.97955 41.97779)
4,Devon Snyder,7967,1064,Addison,SurfaceProblem,41.978218,-87.979507,MMoUtn6D9XovUCMh9vfZgQ,None,2,f,"cracks,grass",None,POINT (-87.97951 41.97822)


In [4]:
#split Mikey and Devon's labels
mikey = chicago_labels[chicago_labels['username'] == 'mikey']
devon = chicago_labels[chicago_labels['username'] == 'Devon Snyder']

In [5]:
def cluster_high_severity_with_the_other(df1,df2,severity_threshold,label_type_id):
    #select only Mikey's labels > 3 for severity
    df1 = df1[df1['severity'] >= severity_threshold]
    # concact with df2
    df = pd.concat([df1,df2])
    label_type_clusters = cluster_label_type_at_index(df,label_type_id)[2]
    label_type_clusters['clustered'] = label_type_clusters.duplicated(subset=['cluster_id']).astype(int)
    #count clustered == 1 and divide by len(df1) 
    print(str(label_types[label_type_id])+ '\n'  + str(label_type_clusters['clustered'].value_counts()))
    print("clustered percentage for df1: " + str(label_type_clusters['clustered'].value_counts()[1]/len(df1)))
    print("clustered percentage for df2: " + str(label_type_clusters['clustered'].value_counts()[1]/len(df2))) 

In [16]:
for i in [2,4,5]:
    cluster_high_severity_with_the_other (mikey,devon,3,i)

Problem
0    194
1     47
Name: clustered, dtype: int64
clustered percentage for df1: 0.8703703703703703
clustered percentage for df2: 0.25133689839572193
SurfaceProblem
0    160
1     20
Name: clustered, dtype: int64
clustered percentage for df1: 0.37037037037037035
clustered percentage for df2: 0.10695187165775401
Obstacle
0    39
1    22
Name: clustered, dtype: int64
clustered percentage for df1: 0.4074074074074074
clustered percentage for df2: 0.11764705882352941


In [12]:
for i in [2,4,5]:
    cluster_high_severity_with_the_other (devon,mikey,3,i)

Problem
0    203
1     33
Name: clustered, dtype: int64
clustered percentage for df1: 0.868421052631579
clustered percentage for df2: 0.16666666666666666
SurfaceProblem
0    171
1      8
Name: clustered, dtype: int64
clustered percentage for df1: 0.21052631578947367
clustered percentage for df2: 0.04040404040404041
Obstacle
0    37
1    20
Name: clustered, dtype: int64
clustered percentage for df1: 0.5263157894736842
clustered percentage for df2: 0.10101010101010101
